In [7]:
# Imports

import urllib.request
import json
from bs4 import BeautifulSoup
import bs4 as bs
import re, requests
import pickle

{'id': '12607212', 't': 'TSLA', 'e': 'NASDAQ', 'l': '347.46', 'l_fix': '347.46', 'l_cur': '347.46', 's': '0', 'ltt': '4:00PM EDT', 'lt': 'Aug 18, 4:00PM EDT', 'lt_dts': '2017-08-18T16:00:01Z', 'c': '-4.46', 'c_fix': '-4.46', 'cp': '-1.27', 'cp_fix': '-1.27', 'ccol': 'chr', 'pcls_fix': '351.92'}
[239000000, 129000000, 124000000]
12.37


In [8]:
# Call this function to give you financial information
def periodic_figure_values(soup, yahoo_figure):

    values = []
    pattern = re.compile(yahoo_figure)

    title = soup.find("strong", text=pattern)    # works for the figures printed in bold
    if title:
        row = title.parent.parent
    else:
        title = soup.find("td", text=pattern)    # works for any other available figure
        if title:
            row = title.parent
        else:
            sys.exit("Invalid figure '" + yahoo_figure + "' passed.")

    cells = row.find_all("td")[1:]    # exclude the <td> with figure name
    for cell in cells:
        if cell.text.strip() != yahoo_figure:    # needed because some figures are indented
            str_value = cell.text.strip().replace(",", "").replace("(", "-").replace(")", "")
            if str_value == "-":
                str_value = 0
            value = int(str_value) * 1000
            values.append(value)

    return values


 

In [9]:
def financials_soup(ticker_symbol, statement="is", quarterly=False):

    if statement == "is" or statement == "bs" or statement == "cf":
        url = "https://finance.yahoo.com/q/" + statement + "?s=" + ticker_symbol
        if not quarterly:
            url += "&annual"
        return BeautifulSoup(requests.get(url).text, "html.parser")

    return exit("Invalid financial statement code '" + statement + "' passed.")

In [10]:
# Use this to get market data for company using symbol
class GoogleFinanceAPI:
    def __init__(self):
        self.prefix = "http://finance.google.com/finance/info?client=ig&q="
    
    def get(self,symbol,exchange):
        url = self.prefix+"%s:%s"%(exchange,symbol)
        u = urllib.request.urlopen(url)
        content = u.read().decode('utf-8')
        obj = json.loads(content[3:])
        return obj[0]

{'id': '12607212', 't': 'TSLA', 'e': 'NASDAQ', 'l': '347.46', 'l_fix': '347.46', 'l_cur': '347.46', 's': '0', 'ltt': '4:00PM EDT', 'lt': 'Aug 18, 4:00PM EDT', 'lt_dts': '2017-08-18T16:00:01Z', 'c': '-4.46', 'c_fix': '-4.46', 'cp': '-1.27', 'cp_fix': '-1.27', 'ccol': 'chr', 'pcls_fix': '351.92'}
[239000000, 129000000, 124000000]
12.37


In [11]:
# This function returns an array of all tickers for publicly traded companies!
def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers


In [ ]:
# Executable code
c = GoogleFinanceAPI()
quote = c.get("TSLA","Nasdaq")
print(quote)
print(periodic_figure_values(financials_soup("NVDA", "is"), "Income Tax Expense"))
save_sp500_tickers()
print (c.get("AMD", "NASDAQ")['l'])
    